# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""17786647240093…","""94500947529846…","""../../../../da…",0.983033,"""29109993339257…","""15243962837121…",false
"""15382455723600…","""10381943658215…","""../../../../da…",0.144221,"""29109993339257…","""17497416807240…",true
"""17842054492291…","""83747384891371…","""../../../../da…",0.679745,"""79120012676251…","""15243962837121…",false
"""21231452742978…","""10381943658215…","""../../../../da…",1.0,"""38621737531313…","""15243962837121…",false
"""14656804312813…","""94500947529846…","""../../../../da…",0.718982,"""38621737531313…","""12289458308824…",true
"""30455791618413…","""83747384891371…","""../../../../da…",0.0,"""63126942124766…",null,false
"""16339131237175…","""83747384891371…","""../../../../da…",0.02333,"""38621737531313…","""12289458308824…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""10631693341967…","""54970406434014…","""../../../../da…",0.983033,"""FirstMortgage3…","""94563687120715…","""Rejected"""
"""46901877029977…","""27176160284815…","""../../../../da…",0.144221,"""FirstMortgage3…","""12997444109085…","""Accepted"""
"""12694450859405…","""16707424364841…","""../../../../da…",0.679745,"""MoneyMarketSav…","""94563687120715…","""Rejected"""
"""17364156545064…","""27176160284815…","""../../../../da…",1.0,"""BasicChecking""","""94563687120715…","""Rejected"""
"""89318631262210…","""54970406434014…","""../../../../da…",0.718982,"""BasicChecking""","""42535379802443…","""Accepted"""
"""45601714038841…","""16707424364841…","""../../../../da…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""11595060206435…","""16707424364841…","""../../../../da…",0.02333,"""BasicChecking""","""42535379802443…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""18178322463443…","""11540291368375…","""../../../../da…",0.983033,"""FirstMortgage3…","""58673201327109…","""Rejected"""
"""74307006442302…","""15334034803640…","""../../../../da…",0.144221,"""FirstMortgage3…","""17403156545851…","""Accepted"""
"""11211069872420…","""16841191183522…","""../../../../da…",0.679745,"""MoneyMarketSav…","""58673201327109…","""Rejected"""
"""10496711704977…","""15334034803640…","""../../../../da…",1.0,"""BasicChecking""","""58673201327109…","""Rejected"""
"""50468173471371…","""11540291368375…","""../../../../da…",0.718982,"""BasicChecking""","""39019879127268…","""Accepted"""
"""87303926616889…","""16841191183522…","""../../../../da…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""95501593358172…","""16841191183522…","""../../../../da…",0.02333,"""BasicChecking""","""39019879127268…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
filename=filename
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""86408357842341…","""62127979447983…",0.983033,"""FirstMortgage3…","""16954231447620…","""Rejected"""
"""58640175940687…","""75446896320649…",0.144221,"""FirstMortgage3…","""16760400260944…","""Accepted"""
"""14817193972399…","""15645869057770…",0.679745,"""MoneyMarketSav…","""16954231447620…","""Rejected"""
"""86320730062727…","""75446896320649…",1.0,"""BasicChecking""","""16954231447620…","""Rejected"""
"""21314812625266…","""62127979447983…",0.718982,"""BasicChecking""","""17719026435943…","""Accepted"""
"""15005592790114…","""15645869057770…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""27618899013807…","""15645869057770…",0.02333,"""BasicChecking""","""17719026435943…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""c73023dda51de1…","""af486bc1815c72…","""../../../../da…",0.983033,"""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""5414b197fb87b3…","""3cdf7b1ae1342e…","""../../../../da…",0.144221,"""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""892f5d61a9215e…","""806ef34eb8237f…","""../../../../da…",0.679745,"""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""c3aa807d9e927a…","""3cdf7b1ae1342e…","""../../../../da…",1.0,"""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""c2d492f9a05073…","""af486bc1815c72…","""../../../../da…",0.718982,"""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""b21059221605c4…","""806ef34eb8237f…","""../../../../da…",0.0,"""UPlusFinPerson…",null,"""Rejected"""
"""80cbe489dcecd4…","""806ef34eb8237f…","""../../../../da…",0.02333,"""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
